## Window Functions in PySpark

Apache Spark window functions let you perform **analytics across related rows** *without collapsing rows* (unlike `groupBy`). They’re essential for rankings, running totals, moving averages, and session-style analysis.





## 1) Define a Window Specification

### What a Window Function Does

A window function computes a value for **each row** based on:

* **Partition** → how rows are grouped
* **Order** → how rows are sorted within each group
* **Frame** → which rows around the current row are considered


In [1]:
import dontno

LAPTOP_IP = dontno.LAPTOP_IP # ← YOUR LAPTOP IP
PC_MASTER = f"spark://{dontno.PC_IP}:7077"

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [3]:

spark = (
    SparkSession.builder
    .appName("DockerRemoteStable")
    .master(PC_MASTER)
    .config("spark.driver.host", LAPTOP_IP)
    .getOrCreate()
)

print("Connected:", spark.version)

Connected: 3.5.8


In [4]:
data = [
    # ---------- IT Department ----------
    (101, "Amit",   "IT",     60000, "2024-01-01"),
    (101, "Amit",   "IT",     62000, "2024-02-01"),
    (101, "Amit",   "IT",     61000, "2024-03-01"),  # salary decrease

    (102, "Ravi",   "IT",     65000, "2024-01-15"),
    (102, "Ravi",   "IT",     65000, "2024-02-15"),  # same salary (tie)
    (102, "Ravi",   "IT",     67000, "2024-03-15"),

    (103, "Sneha",  "IT",     65000, "2024-02-01"),  # joins late
    (103, "Sneha",  "IT",     None,  "2024-03-01"),  # NULL salary
    (103, "Sneha",  "IT",     68000, "2024-04-01"),

    # ---------- HR Department ----------
    (104, "Kiran",  "HR",     45000, "2024-01-01"),
    (104, "Kiran",  "HR",     46000, "2024-02-01"),

    (105, "Pooja",  "HR",     48000, "2024-01-01"),
    (105, "Pooja",  "HR",     48000, "2024-03-01"),  # missing Feb
    (105, "Pooja",  "HR",     50000, "2024-04-01"),

    (106, "Neha",   "HR",     48000, "2024-02-01"),  # single record

    # ---------- Sales Department ----------
    (107, "Arjun",  "Sales",  55000, "2024-01-01"),
    (107, "Arjun",  "Sales",  53000, "2024-02-01"),  # decrease
    (107, "Arjun",  "Sales",  56000, "2024-03-01"),

    (108, "Manoj",  "Sales",  52000, "2024-01-15"),
    (108, "Manoj",  "Sales",  52000, "2024-02-15"),  # tie
    (108, "Manoj",  "Sales",  52000, "2024-03-15"),  # repeated tie

    (109, "Divya",  "Sales",  58000, "2024-02-01"),  # joins mid
    (109, "Divya",  "Sales",  60000, "2024-03-01"),

    # ---------- Finance (single-row partition) ----------
    (110, "Rahul",  "Finance",70000, "2024-03-01")
]


In [8]:
from pyspark.sql.types import StringType, StructField, StructType, IntegerType

company_schema = StructType([
    StructField("emp_id", IntegerType()),
    StructField("name", StringType()),
    StructField("department", StringType()),
    StructField("salary", IntegerType()),
    StructField("salary_date", StringType())
])

In [9]:
company_df = spark.createDataFrame(data, schema=company_schema)

In [11]:
from pyspark.sql.functions import to_date


company_df = company_df.withColumn("salary_date", to_date("salary_date")).sort("salary", ascending=False)


In [12]:
company_df.show()

+------+-----+----------+------+-----------+
|emp_id| name|department|salary|salary_date|
+------+-----+----------+------+-----------+
|   110|Rahul|   Finance| 70000| 2024-03-01|
|   103|Sneha|        IT| 68000| 2024-04-01|
|   102| Ravi|        IT| 67000| 2024-03-15|
|   103|Sneha|        IT| 65000| 2024-02-01|
|   102| Ravi|        IT| 65000| 2024-01-15|
|   102| Ravi|        IT| 65000| 2024-02-15|
|   101| Amit|        IT| 62000| 2024-02-01|
|   101| Amit|        IT| 61000| 2024-03-01|
|   109|Divya|     Sales| 60000| 2024-03-01|
|   101| Amit|        IT| 60000| 2024-01-01|
|   109|Divya|     Sales| 58000| 2024-02-01|
|   107|Arjun|     Sales| 56000| 2024-03-01|
|   107|Arjun|     Sales| 55000| 2024-01-01|
|   107|Arjun|     Sales| 53000| 2024-02-01|
|   108|Manoj|     Sales| 52000| 2024-01-15|
|   108|Manoj|     Sales| 52000| 2024-02-15|
|   108|Manoj|     Sales| 52000| 2024-03-15|
|   105|Pooja|        HR| 50000| 2024-04-01|
|   105|Pooja|        HR| 48000| 2024-01-01|
|   105|Po

In [13]:
company_df.groupBy("department").sum("salary").show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|        IT|     513000|
|     Sales|     438000|
|        HR|     285000|
|   Finance|      70000|
+----------+-----------+



Optionally add a **frame**:

In [14]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy("department").orderBy("salary")

---

## 2) Common Window Functions

### Ranking Functions

In [15]:
from pyspark.sql.functions import row_number,rank, dense_rank

company_df = company_df.withColumn("row_num", row_number().over(window=window_spec))
company_df.show()

+------+-----+----------+------+-----------+-------+
|emp_id| name|department|salary|salary_date|row_num|
+------+-----+----------+------+-----------+-------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|
|   106| Neha|        HR| 48000| 2024-02-01|      5|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|
|   103|Sneha|        IT|  NULL| 2024-03-01|      1|
|   101| Amit|        IT| 60000| 2024-01-01|      2|
|   101| Amit|        IT| 61000| 2024-03-01|      3|
|   101| Amit|        IT| 62000| 2024-02-01|      4|
|   103|Sneha|        IT| 65000| 2024-02-01|      5|
|   102| Ravi|        IT| 65000| 2024-01-15|      6|
|   102| Ravi|        IT| 65000| 2024-02-15|      7|
|   102| Ravi|        IT| 67000| 2024-03-15|      8|
|   103|Sneha|        IT| 68000| 2024-04-01|  

In [16]:
company_df = company_df.withColumn("rank", rank().over(window=window_spec))
company_df.show()

+------+-----+----------+------+-----------+-------+----+
|emp_id| name|department|salary|salary_date|row_num|rank|
+------+-----+----------+------+-----------+-------+----+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|
|   105|Pooja|        HR| 48000| 2024-03-01|      3|   3|
|   106| Neha|        HR| 48000| 2024-02-01|      4|   3|
|   105|Pooja|        HR| 48000| 2024-01-01|      5|   3|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|
|   103|Sneha|        IT|  NULL| 2024-03-01|      1|   1|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|
|   101| Amit|        IT| 61000| 2024-03-01|      3|   3|
|   101| Amit|        IT| 62000| 2024-02-01|      4|   4|
|   102| Ravi|        IT| 65000| 2024-01-15|      5|   5|
|   102| Ravi|        IT| 65000| 2024-02-15|      6|   5|
|   103|Sneha|        IT| 65000| 2024-02-01|      7|   5|
|   102| Ravi|

In [17]:
company_df = company_df.withColumn("dense_rank", dense_rank().over(window=window_spec))
company_df.show()

+------+-----+----------+------+-----------+-------+----+----------+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|
+------+-----+----------+------+-----------+-------+----+----------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|
|   105|Pooja|        HR| 48000| 2024-03-01|      3|   3|         3|
|   106| Neha|        HR| 48000| 2024-02-01|      4|   3|         3|
|   105|Pooja|        HR| 48000| 2024-01-01|      5|   3|         3|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|
|   103|Sneha|        IT|  NULL| 2024-03-01|      1|   1|         1|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|
|   101| Amit|        IT| 61000| 2024-03-01|      3|   3|         3|
|   101| Amit|        IT| 62000| 2024-02-01|      4|   4|         4|
|   102| Ravi|        IT| 65000| 2

**Differences**

* `row_number()` → unique sequence (no ties)
* `rank()` → gaps after ties
* `dense_rank()` → no gaps

![Image](https://miro.medium.com/1%2AtuGFvhwk5rUtoQWcX4A6ng.gif)

![Image](https://media.licdn.com/dms/image/v2/D4D12AQFRS0AU_T_NQQ/article-cover_image-shrink_720_1280/article-cover_image-shrink_720_1280/0/1654768699532?e=2147483647\&t=ONuIV9v8k99yhD5U52ocS8i3WOsKk_p_xlfjmGVVXEg\&v=beta)

---

### Aggregate Functions (Windowed)

In [18]:
company_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Window [row_number() windowspecdefinition(department#2, salary#3 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS row_num#59, rank(salary#3) windowspecdefinition(department#2, salary#3 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank#93, dense_rank(salary#3) windowspecdefinition(department#2, salary#3 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS dense_rank#138], [department#2], [salary#3 ASC NULLS FIRST]
   +- Sort [department#2 ASC NULLS FIRST, salary#3 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(department#2, 200), ENSURE_REQUIREMENTS, [plan_id=427]
         +- Sort [salary#3 DESC NULLS LAST], true, 0
            +- Exchange rangepartitioning(salary#3 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=424]
               +- Project [emp_id#0, name#1, department#2, salary#3, ca

In [19]:
company_df.show()

+------+-----+----------+------+-----------+-------+----+----------+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|
+------+-----+----------+------+-----------+-------+----+----------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|
|   105|Pooja|        HR| 48000| 2024-03-01|      3|   3|         3|
|   106| Neha|        HR| 48000| 2024-02-01|      4|   3|         3|
|   105|Pooja|        HR| 48000| 2024-01-01|      5|   3|         3|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|
|   103|Sneha|        IT|  NULL| 2024-03-01|      1|   1|         1|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|
|   101| Amit|        IT| 61000| 2024-03-01|      3|   3|         3|
|   101| Amit|        IT| 62000| 2024-02-01|      4|   4|         4|
|   102| Ravi|        IT| 65000| 2

In [20]:
from pyspark.sql.functions import sum, avg, min, max

In [21]:
company_df.withColumn("avg_salary", avg("salary").over(Window.partitionBy("department").orderBy("salary_date"))).show()

+------+-----+----------+------+-----------+-------+----+----------+-----------------+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|       avg_salary|
+------+-----+----------+------+-----------+-------+----+----------+-----------------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|          70000.0|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|          46500.0|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|   3|         3|          46500.0|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|          46750.0|
|   106| Neha|        HR| 48000| 2024-02-01|      5|   3|         3|          46750.0|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|   3|         3|          47000.0|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|          47500.0|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|          60000.0|
|   102| Ravi|        IT| 65000| 2024-01-15

In [22]:
company_df.withColumn("sum_salary", sum("salary").over(Window.partitionBy("department").orderBy("salary_date"))).show()

+------+-----+----------+------+-----------+-------+----+----------+----------+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|sum_salary|
+------+-----+----------+------+-----------+-------+----+----------+----------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|     70000|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|     93000|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|   3|         3|     93000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|    187000|
|   106| Neha|        HR| 48000| 2024-02-01|      5|   3|         3|    187000|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|   3|         3|    235000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|    285000|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|     60000|
|   102| Ravi|        IT| 65000| 2024-01-15|      6|   5|         5|    125000|
|   101| Amit|        IT| 62000| 2024-02

In [23]:
company_df.withColumn("min_salary", min("salary").over(Window.partitionBy("department"))).show()

+------+-----+----------+------+-----------+-------+----+----------+----------+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|min_salary|
+------+-----+----------+------+-----------+-------+----+----------+----------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|     70000|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|     45000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|     45000|
|   105|Pooja|        HR| 48000| 2024-03-01|      3|   3|         3|     45000|
|   106| Neha|        HR| 48000| 2024-02-01|      4|   3|         3|     45000|
|   105|Pooja|        HR| 48000| 2024-01-01|      5|   3|         3|     45000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|     45000|
|   103|Sneha|        IT|  NULL| 2024-03-01|      1|   1|         1|     60000|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|     60000|
|   101| Amit|        IT| 61000| 2024-03

In [24]:
company_df.withColumn("max_salary", max("salary").over(Window.partitionBy("department"))).show()

+------+-----+----------+------+-----------+-------+----+----------+----------+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|max_salary|
+------+-----+----------+------+-----------+-------+----+----------+----------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|     70000|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|     50000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|     50000|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|   3|         3|     50000|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|   3|         3|     50000|
|   106| Neha|        HR| 48000| 2024-02-01|      5|   3|         3|     50000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|     50000|
|   103|Sneha|        IT|  NULL| 2024-03-01|      1|   1|         1|     68000|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|     68000|
|   101| Amit|        IT| 61000| 2024-03

> Same aggregates as `groupBy`, but **row-level output is preserved**.

---

### Analytical / Value Functions

In [25]:
from pyspark.sql.functions import lag, lead, first, last

company_df.withColumn("lag", lag("salary", 1).over(Window.partitionBy("department").orderBy("salary_date"))).show()

+------+-----+----------+------+-----------+-------+----+----------+-----+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|  lag|
+------+-----+----------+------+-----------+-------+----+----------+-----+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1| NULL|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1| NULL|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|   3|         3|45000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|48000|
|   106| Neha|        HR| 48000| 2024-02-01|      5|   3|         3|46000|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|   3|         3|48000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|48000|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2| NULL|
|   102| Ravi|        IT| 65000| 2024-01-15|      6|   5|         5|60000|
|   101| Amit|        IT| 62000| 2024-02-01|      4|   4|         4|65000|
|   103|Sneha|        IT|

In [26]:
company_df.withColumn("lead", lead("salary", 1).over(Window.partitionBy("department").orderBy("salary_date"))).show()

+------+-----+----------+------+-----------+-------+----+----------+-----+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank| lead|
+------+-----+----------+------+-----------+-------+----+----------+-----+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1| NULL|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|48000|
|   105|Pooja|        HR| 48000| 2024-01-01|      5|   3|         3|46000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|48000|
|   106| Neha|        HR| 48000| 2024-02-01|      4|   3|         3|48000|
|   105|Pooja|        HR| 48000| 2024-03-01|      3|   3|         3|50000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4| NULL|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|65000|
|   102| Ravi|        IT| 65000| 2024-01-15|      5|   5|         5|62000|
|   101| Amit|        IT| 62000| 2024-02-01|      4|   4|         4|65000|
|   103|Sneha|        IT|

In [27]:
company_df.withColumn("first", first("salary").over(Window.partitionBy("department").orderBy("salary_date"))).show()

+------+-----+----------+------+-----------+-------+----+----------+-----+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|first|
+------+-----+----------+------+-----------+-------+----+----------+-----+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|70000|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|45000|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|   3|         3|45000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|45000|
|   106| Neha|        HR| 48000| 2024-02-01|      5|   3|         3|45000|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|   3|         3|45000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|45000|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|60000|
|   102| Ravi|        IT| 65000| 2024-01-15|      6|   5|         5|60000|
|   101| Amit|        IT| 62000| 2024-02-01|      4|   4|         4|60000|
|   103|Sneha|        IT|

In [28]:
company_df.withColumn("last", last("salary").over(Window.partitionBy("department").orderBy("salary_date"))).show()

+------+-----+----------+------+-----------+-------+----+----------+-----+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank| last|
+------+-----+----------+------+-----------+-------+----+----------+-----+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|70000|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|48000|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|   3|         3|48000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|48000|
|   106| Neha|        HR| 48000| 2024-02-01|      5|   3|         3|48000|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|   3|         3|48000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|50000|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|60000|
|   102| Ravi|        IT| 65000| 2024-01-15|      6|   5|         5|65000|
|   101| Amit|        IT| 62000| 2024-02-01|      4|   4|         4|65000|
|   103|Sneha|        IT|

Use `ignoreNulls=True` when needed:

---

## 3) Rows vs Range Frames (Very Important)

### Rows-based frame

In [29]:
win_spec = Window.orderBy("salary_date").rowsBetween(-2, 0)

* Counts **physical rows**
* Deterministic with strict ordering

### Range-based frame

In [30]:
Window.orderBy("date").rangeBetween(-7, 0)

* Uses **value ranges**
* Multiple rows can share the same order key

> For time series with duplicate timestamps, **rowsBetween** is safer.

---

## 4) Practical Data Engineering Examples

### Latest Record per Key (De-duplication)

In [35]:
from pyspark.sql.functions import col, lit, when

w = Window.partitionBy("department").orderBy(col("salary_date").desc())

company_df.withColumn("rn", row_number().over(w)).filter("rn == 1").show()

+------+-----+----------+------+-----------+-------+----+----------+---+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank| rn|
+------+-----+----------+------+-----------+-------+----+----------+---+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|  1|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|  1|
|   103|Sneha|        IT| 68000| 2024-04-01|      9|   9|         7|  1|
|   108|Manoj|     Sales| 52000| 2024-03-15|      3|   1|         1|  1|
+------+-----+----------+------+-----------+-------+----+----------+---+



### Top-N per Group

In [37]:
w = Window.partitionBy("department").orderBy(col("salary_date").desc())

company_df.withColumn("dr", dense_rank().over(w)).filter("dr <= 3").show()

+------+-----+----------+------+-----------+-------+----+----------+---+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank| dr|
+------+-----+----------+------+-----------+-------+----+----------+---+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|  1|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|  1|
|   105|Pooja|        HR| 48000| 2024-03-01|      3|   3|         3|  2|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|  3|
|   106| Neha|        HR| 48000| 2024-02-01|      4|   3|         3|  3|
|   103|Sneha|        IT| 68000| 2024-04-01|      9|   9|         7|  1|
|   102| Ravi|        IT| 67000| 2024-03-15|      8|   8|         6|  2|
|   103|Sneha|        IT|  NULL| 2024-03-01|      1|   1|         1|  3|
|   101| Amit|        IT| 61000| 2024-03-01|      3|   3|         3|  3|
|   108|Manoj|     Sales| 52000| 2024-03-15|      3|   1|         1|  1|
|   107|Arjun|     Sales| 56000| 2024-03-01|      6

### Running Metrics (Finance / Logs)

In [38]:
w = Window.partitionBy("department").orderBy("salary_date") \
          .rowsBetween(Window.unboundedPreceding, Window.currentRow)

company_df.withColumn("total_salary", sum("salary").over(w)).show()

+------+-----+----------+------+-----------+-------+----+----------+------------+
|emp_id| name|department|salary|salary_date|row_num|rank|dense_rank|total_salary|
+------+-----+----------+------+-----------+-------+----+----------+------------+
|   110|Rahul|   Finance| 70000| 2024-03-01|      1|   1|         1|       70000|
|   104|Kiran|        HR| 45000| 2024-01-01|      1|   1|         1|       45000|
|   105|Pooja|        HR| 48000| 2024-01-01|      3|   3|         3|       93000|
|   104|Kiran|        HR| 46000| 2024-02-01|      2|   2|         2|      139000|
|   106| Neha|        HR| 48000| 2024-02-01|      5|   3|         3|      187000|
|   105|Pooja|        HR| 48000| 2024-03-01|      4|   3|         3|      235000|
|   105|Pooja|        HR| 50000| 2024-04-01|      6|   6|         4|      285000|
|   101| Amit|        IT| 60000| 2024-01-01|      2|   2|         2|       60000|
|   102| Ravi|        IT| 65000| 2024-01-15|      6|   5|         5|      125000|
|   101| Amit|  

---

## 5) Performance Notes (Critical for Interviews)

* Window functions **cause shuffles** if `partitionBy` changes data distribution
* Prefer **low-cardinality partitions**
* Avoid very wide frames on huge datasets
* Cache if the same window spec is reused
* Window ≠ groupBy: window keeps **N rows**, groupBy reduces to **K rows**

---

## Quick Mental Model

| Aspect         | groupBy | Window |
| -------------- | ------- | ------ |
| Rows preserved | ❌       | ✅      |
| Aggregation    | ✅       | ✅      |
| Ranking / Lag  | ❌       | ✅      |
| Shuffle        | Yes     | Yes    |

---

If you want, I can next:

* Deep-dive into **window execution internals (DAG + shuffle)**
* Compare **SQL window functions vs PySpark**
* Give **interview-grade window function questions**
* Optimize a **real dataset window query** you’re working on

In [ ]:
import pathlib

filepath = pathlib.Path().cwd()

filepath

In [ ]:
from google.colab import drive
drive.mount('/content/drive')